<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/SoundOn_Podcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><strong>SoundOn Podcast 爬蟲(已失效)</strong></h1>

<p>爬取SoundOn上的Podcast，只要輸入頻道的網址，便能擷取所有的單集，並包含以下資訊：出刊日、標題、長度、單集連結、音訊連結、單集摘要</p>

<p><strong>可同時下載各集音檔</strong></p>


# 頻道網址與檔名設定

In [ ]:
url = 'https://player.soundon.fm/p/e75f72d6-a458-4982-861e-6c5fbad85956' #財報狗
#url = 'https://player.soundon.fm/p/d2aab16c-3a70-4023-b52b-e50f07852ecd' #財經M平方
#url = 'https://player.soundon.fm/p/954689a5-3096-43a4-a80b-7810b219cef3' #Gooaye 股癌

file_name = '財報狗'


# 安裝並載入套件

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By



!pip install fake_useragent
from fake_useragent import UserAgent
from lxml import etree
import requests
import pandas as pd
import random
import time

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

#瀏覽器參數設定

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


#載入網頁(頻道)

In [ ]:
#下載網頁
browser.get(url)
for i in range(200):

  browser.execute_script("window.scrollBy(0, window.innerHeight)")
  t = random.uniform(0.5, 1.5) #隨機數
  time.sleep(t) #暫停


#擷取網頁資訊

In [ ]:
#擷取頻道目錄
channel_name = [] #裝頻道名稱
title = [] #裝標題
date = [] #裝日期
links = [] #裝連結
time_long = [] #裝時間長度

#擷取網頁元素
temp_channel = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[1]/div/div[2]/div[2]/h3')[0].text
URLs = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a')
temp_title = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a/div/div[2]/h4/div')
temp_date = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a/div/div[2]/div[2]/time[2]')
temp_time = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a/div/div[2]/div[2]/time[1]') 
#/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div/a/div/div[2]/div[2]/time[1]/text()
#擷取連結
for node in URLs:
  links.append(node.get_attribute("href"))
links = pd.DataFrame(links)

#擷取標題、出刊日、長度
t = len(URLs)
for i in range(len(URLs)):
  channel_name.append(temp_channel)
  date.append(temp_date[i].text)
  title.append(temp_title[i].text)
  time_long.append(str(temp_time[i].text))



#整理為表格
channel_name = pd.DataFrame(channel_name)
date = pd.DataFrame(date)
title = pd.DataFrame(title)
time_long = pd.DataFrame(time_long)

data = pd.concat([channel_name, date, title, time_long, links], axis=1)
data.columns = ['channel', 'date', 'title', 'long', 'link']

In [ ]:
#擷取單集摘要、音訊網址
audio_link = []
Summary = []

for i in range(len(data['link'])):

  url = data['link'][i]
  #下載網頁
  browser.get(url)
  t = random.uniform(0.75, 1.5) #隨機數
  time.sleep(t) #暫停

  #取得播放連結
  play = browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[2]/div/div[1]/div/div[3]/div/button')
  play.click()
  browser.implicitly_wait(0.05) #等待載入
  element = browser.find_elements_by_xpath('/html/body/audio')

  temp_audio = []
  for node in element:
    temp_audio.append(node.get_attribute("src"))
  audio_link.append(temp_audio)


  #取得摘要
  temp_Summary = browser.find_elements(By.XPATH,'/html/body/div[1]/div/div[1]/div[2]/div/div[2]/div/div')[0].text
  temp_Summary = temp_Summary.replace('\n', '')
  Summary.append(temp_Summary)

audio_link = pd.DataFrame(audio_link)
Summary = pd.DataFrame(Summary)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead


#檢視資料

In [ ]:
data


,channel,date,title,long,link
0,財報狗 - 掌握台股美股時事議題,2022-04-05,110.【台股美股提款機】無懼消費市場轉弱 伺服器可望引領今年美光業績創新高,26:26,https://player.soundon.fm/p/e75f72d6-a458-4982...
1,財報狗 - 掌握台股美股時事議題,2022-03-31,109.【財經時事放大鏡】疫情帶動外送風潮也枉然 雲端廚房的經營難題多,22:48,https://player.soundon.fm/p/e75f72d6-a458-4982...
2,財報狗 - 掌握台股美股時事議題,2022-03-29,108.【台股美股提款機】記憶體產業風水輪流轉 NAND Flash 未來兩季將優於 DRAM,39:21,https://player.soundon.fm/p/e75f72d6-a458-4982...
3,財報狗 - 掌握台股美股時事議題,2022-03-24,107.【財經時事放大鏡】高階顯卡報價跳水 俄烏戰後的高糧價風暴難解,21:56,https://player.soundon.fm/p/e75f72d6-a458-4982...
4,財報狗 - 掌握台股美股時事議題,2022-03-20,106.【達人聊產業】伺服器產業揭密 ＆ 相關投資機會大盤點,49:14,https://player.soundon.fm/p/e75f72d6-a458-4982...
...,...,...,...,...,...
116,財報狗 - 掌握台股美股時事議題,2020-06-18,真・佛系投資人 - 華爾街股票研究員 Vincent Ep. #4,49:42,https://player.soundon.fm/p/e75f72d6-a458-4982...
117,財報狗 - 掌握台股美股時事議題,2020-06-10,股票、期權、賭博與詐騙 - 麥克風的市場求生手冊 Ep. #3,51:26,https://player.soundon.fm/p/e75f72d6-a458-4982...
118,財報狗 - 掌握台股美股時事議題,2020-05-28,29 歲如何管理 10 億資金 - 博弈式價值投資人 Frank Ep. #2,49:18,https://player.soundon.fm/p/e75f72d6-a458-4982...
119,財報狗 - 掌握台股美股時事議題,2020-05-21,財報狗共同創辦人 Jeff - 從財報背後挖出成長股 Ep. #1,55:30,https://player.soundon.fm/p/e75f72d6-a458-4982...


#匯出資料

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


data = pd.concat([channel_name, date, title, time_long, links, audio_link, Summary], axis=1)
data.columns = ['channel', 'date', 'title', 'long', 'link', 'audio_link', 'Summary']


#輸出檔案
path = '/content/drive/MyDrive/Data Set/'+file_name+'.csv' #路徑
data.to_csv(path, index=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#下載Podcast音檔

In [ ]:
for i in range(len(data['link'])):

  url = data['audio_link'][i]

  #設定檔名
  file_name = data['channel'][i] + ' ' + data['date'][i]
  downloaded_file_location = '/content/drive/MyDrive/Data Set/Podcast/'+file_name+'.mp3'

  # Downloading the audio file
  headers = UserAgent().random  #隨機請求表頭
  req = requests.get(url, headers={"User-Agent":headers}).content

  t = random.uniform(0.5, 1) #隨機數
  time.sleep(t) #暫停

  with open(downloaded_file_location, 'wb') as f:
      f.write(req)